# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
# create square root generator and calculate sum of the outputs of the generator for limt 5
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

limit = 5
generator = square_root_generator(limit)

sum = 0
for sqrt_value in generator:
    sum = sum + sqrt_value
print('The sum of the outputs of the generator for limit = 5: ', sum)



The sum of the outputs of the generator for limit = 5:  8.382332347441762


In [2]:
# Now set limit as 13 and check 13th number yielded bt the generator
limit = 13
generator = square_root_generator(limit)

print('13th number yielded is:',list(generator)[-1])



13th number yielded is: 3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [3]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [4]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [5]:
# import the libraries
import dlt
import duckdb

In [6]:
# run dlt pipeline to load records from generator 1
# create the pipeline
pipeline = dlt.pipeline(
						destination='duckdb',
						dataset_name='people')

# run the pipeline with default settings, and capture the outcome
info = pipeline.run(people_1(),
                    table_name="persons",
                    write_disposition="replace")

# show the outcome

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.38 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708214746.3128371 is LOADED and contains no failed jobs


In [7]:
# check tables loaded and return records from the loaded table
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

persons = conn.sql("SELECT * FROM persons").df()
display(persons)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ persons             │
│ users               │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1708214746.3128371,Njd6VIYHHfGSPA
1,2,Person_2,27,City_A,1708214746.3128371,JJiyUpjlBNCInA
2,3,Person_3,28,City_A,1708214746.3128371,4uZ1M0N0vptgVA
3,4,Person_4,29,City_A,1708214746.3128371,cpRR6P+1YSIgIw
4,5,Person_5,30,City_A,1708214746.3128371,oB+x5WA87mhhLA


In [8]:
# check sum of all the ages in the table
age_sum = conn.sql("SELECT SUM(age) FROM persons").df()
print(age_sum)

   sum(age)
0     140.0


In [9]:
# run dlt pipeline to append records from generator 2
info = pipeline.run(people_2(),
                        table_name="persons",
                        write_disposition="append")
# show the outcome
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.24 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708214747.456533 is LOADED and contains no failed jobs


In [10]:
# check tables loaded and return records from the updated table
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

persons = conn.sql("SELECT * FROM persons").df()
display(persons)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ persons             │
│ users               │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708214746.3128371,Njd6VIYHHfGSPA,None
1,2,Person_2,27,City_A,1708214746.3128371,JJiyUpjlBNCInA,None
2,3,Person_3,28,City_A,1708214746.3128371,4uZ1M0N0vptgVA,None
3,4,Person_4,29,City_A,1708214746.3128371,cpRR6P+1YSIgIw,None
4,5,Person_5,30,City_A,1708214746.3128371,oB+x5WA87mhhLA,None
5,3,Person_3,33,City_B,1708214747.456533,zpjXWpRGkdbS3g,Job_3
6,4,Person_4,34,City_B,1708214747.456533,CNQ6O0laYRqovg,Job_4
7,5,Person_5,35,City_B,1708214747.456533,0uJn8sVAB3jW0g,Job_5
8,6,Person_6,36,City_B,1708214747.456533,ipdB+XpD08ZXMA,Job_6
9,7,Person_7,37,City_B,1708214747.456533,lFbo37rqBfbXtw,Job_7


In [11]:
# check sum of all the ages in the table
age_sum = conn.sql("SELECT SUM(age) FROM persons").df()
print(age_sum)

   sum(age)
0     353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [12]:
# run dlt pipeline to load records from generator 1
# create the pipeline

pipeline = dlt.pipeline(
						destination='duckdb',
						dataset_name='people')

# run the pipeline with default settings, and capture the outcome
info = pipeline.run(people_1(),
                    table_name="people",
                    write_disposition="replace", primary_key='id')

# show the outcome

print(info)


Pipeline dlt_colab_kernel_launcher load step completed in 0.26 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708214748.3100128 is LOADED and contains no failed jobs


In [13]:
# check tables loaded and return records from the loaded table
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

people = conn.sql("SELECT * FROM people").df()
display(people)


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ persons             │
│ users               │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708214748.3100128,a2FHGA+42dkybQ,None
1,2,Person_2,27,City_A,1708214748.3100128,bRkMc0DjlbK8bA,None
2,3,Person_3,28,City_A,1708214748.3100128,aq/V00J9+wQgNw,None
3,4,Person_4,29,City_A,1708214748.3100128,ieWC7f3YDQI5JA,None
4,5,Person_5,30,City_A,1708214748.3100128,nnQ+WSgaAkUYjw,None


In [14]:
# run dlt pipeline to merge records from generator 2
info = pipeline.run(people_2(), table_name="people", write_disposition="merge", primary_key="id")

# show the outcome
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.43 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708214749.0908184 is LOADED and contains no failed jobs


In [15]:
# check tables loaded and return records from the updated table
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

people = conn.sql("SELECT * FROM people").df()
display(people)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ persons             │
│ users               │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708214748.3100128,a2FHGA+42dkybQ,None
1,2,Person_2,27,City_A,1708214748.3100128,bRkMc0DjlbK8bA,None
2,8,Person_8,38,City_B,1708214749.0908184,HBhIjcHYD+kR1w,Job_8
3,4,Person_4,34,City_B,1708214749.0908184,2jBF/2guoZvG3A,Job_4
4,5,Person_5,35,City_B,1708214749.0908184,VN6I0A36GUKptw,Job_5
5,7,Person_7,37,City_B,1708214749.0908184,9Rb8IpmcRQfbBw,Job_7
6,3,Person_3,33,City_B,1708214749.0908184,9EFQeiNsI0d+4A,Job_3
7,6,Person_6,36,City_B,1708214749.0908184,JKgKD7lPHBhEfg,Job_6


In [16]:
# check sum of all the ages in the table
age_sum = conn.sql("SELECT SUM(age) FROM people").df()
print(age_sum)

   sum(age)
0     266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX